This is the baseline for the project. This will be implemented as various classifier models over the 29,950 training instances (15k fire = 0, 14,950 fire = 1).

In [1]:
import pandas as pd
import os
import csv
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Change directory to where you want files saved
os.chdir('/content/sample_data')


# File containing all data
data = pd.read_csv("/content/combined_data.csv")

X = pd.DataFrame(data)

# Drop the response from X
X.drop("FIRE", axis = 1, inplace = True)
X.drop("Date", axis = 1, inplace = True)

# Assign y to the 'FIRE' column in data
y = data['FIRE']

# Set seed for reproducability
X_train, X_test, y_train, y_test, data_train, data_test = train_test_split(X, y, data, test_size = 0.2, random_state = 1234)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/content/combined_data.csv'

In [2]:
import pandas as pd
import os
import csv
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt




# File containing all data
data = pd.read_csv("combined_data.csv")

data = data.rename(columns={'Latitude': 'temp', 'Longitude': 'Latitude'}) # switching lat and long columns. These got mixed up in original dataset, so want to make sure they are correct here.
data = data.rename(columns={'temp': 'Longitude'})


X = pd.DataFrame(data)


# Drop the response from X
X.drop("FIRE", axis = 1, inplace = True)
X.drop("Date", axis = 1, inplace = True)

# Assign y to the 'FIRE' column in data
y = data['FIRE']

# Set seed for reproducability
X_train_all, X_test, y_train_all, y_test, data_train_all, data_test = train_test_split(X, y, data, test_size = 0.2, random_state = 1234)

#create validation set
X_train, X_val, y_train, y_val, data_train, data_val = train_test_split(X_train_all, y_train_all, data_train_all, test_size = 0.2, random_state = 1234)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ALDA Project/Data'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier as SKKerasClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (accuracy_score, precision_score,
                            recall_score, f1_score, classification_report,
                            confusion_matrix, ConfusionMatrixDisplay)

# Evaluates pipelines passed
def evaluate_pipelines(pipelines, X_train, y_train, X_evaluate, y_evaluate):
    rows = []
    for name, pipeline in pipelines:
        pipeline.fit(X_train, y_train)

        train_preds = pipeline.predict(X_train)
        test_preds = pipeline.predict(X_test)

        training_accuracy = accuracy_score(y_train, train_preds)
        testing_accuracy = accuracy_score(y_test, test_preds)

        training_recall = recall_score(y_train, train_preds)
        testing_recall = recall_score(y_test, test_preds)

        training_precision = precision_score(y_train, train_preds)
        testing_precision = precision_score(y_test, test_preds)

        training_f1 = f1_score(y_train, train_preds)
        testing_f1 = f1_score(y_test, test_preds)

        rows.append({'name': name, 'Training Accuracy': training_accuracy, 'Eval Accuracy': testing_accuracy,
                     'Training Recall': training_recall, 'Eval Recall': testing_recall,
                     'Training Precision': training_precision, 'Eval Precision': testing_precision,
                     'Training F1': training_f1, 'Eval F1': testing_f1})

    display(pd.DataFrame(rows))

# Plots confusion matrix
def plot_confusion_matrix(y_true, y_pred, clf_name):
    # Generate confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Create figure
    plt.figure()

    # Use seaborn to plot confusion matrix
    sns.heatmap(cm, annot = True, fmt = 'g', cmap='viridis')

    # Labels/title
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'{clf_name} Confusion Matrix')

    # Display plot
    plt.show()


# Define ANN architecture
def build_ann():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


random_forest = make_pipeline(RandomForestClassifier())
decision_tree = make_pipeline(DecisionTreeClassifier())
logistic_regression = make_pipeline(LogisticRegression())
multilayer_perceptron = make_pipeline(MLPClassifier(max_iter = 1000))
keras_ann = make_pipeline(
    StandardScaler(),
    SKKerasClassifier(model=build_ann, epochs=50, batch_size=32, verbose=0)
)



evaluate_pipelines([
    ('Random Forest', random_forest),
    ('Decision Tree', decision_tree),
    ('Logistic Regression', logistic_regression),
    ('Multilayer Perceptron', multilayer_perceptron),
    ('Keras ANN', keras_ann)
], X_train, y_train, X_test, y_test)

# CLFs to plot confusion matrix for
clfs = [('Random Forest', random_forest),
        ('Decision Tree', decision_tree),
        ('Logistic Regression', logistic_regression),
        ('Multilayer Perceptron', multilayer_perceptron),
         ('Keras ANN', keras_ann)
        ]

for name, clf in clfs:
    print(f"\nTraining {name}...")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    plot_confusion_matrix(y_test, y_pred, name)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: 'super' object has no attribute '__sklearn_tags__'